In [1]:
pip install PyPDF2 faiss-cpu tiktoken

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 232 kB 407 kB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 322 kB/s eta 0:00:011
     |████████████████████████████████| 1.0 MB 314 kB/s eta 0:00:01
     |████████████████████████████████| 284 kB 409 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

/Users/pathaoltd/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import os
from constants import openai_key

os.environ["OPENAI_API_KEY"] = openai_key

In [4]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('Paper1.pdf')

In [5]:
from typing_extensions import Concatenate

# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
raw_text

'BRIDGING CLASSICAL AND QUANTUM MACHINE LEARNING : KNOWLEDGE\nTRANSFER FROM CLASSICAL TO QUANTUM NEURAL NETWORKS USING\nKNOWLEDGE DISTILLATION\nRESEARCH ARTICLE\nMohammad Junayed Hasan1and M.R.C. Mahdy1, *\n1Department of Electrical and Computer Engineering, North South University, Bashundhara, Dhaka\nABSTRACT\nVery recently, studies have shown that quantum neural networks surpass classical neural networks in\ntasks like image classification when a similar number of learnable parameters are used. However,\nthe development and optimization of quantum models are currently hindered by issues such as qubit\ninstability and limited qubit availability, leading to error-prone systems with weak performance.\nOn the other hand, classical models can exhibit high-performance owing to substantial resource\navailability. As a result, more studies have been focusing on hybrid classical-quantum integration to\nleverage the advantages of both paradigms. A line of research particularly focuses on trans

In [7]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [8]:
len(texts)

122

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
document_search = FAISS.from_texts(texts, embeddings)

In [ ]:
document_search

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "What are the main research questions of this study?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)